## Trabalho 3 
### Exercicio 1



O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

``` python
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1
    while r_ != 0
      q = r div r_
      r, r_, s, s_, t, t_ = r_, r − q × r_, s_, s − q × s_, t_, t − q × t_
    OUTPUT r, s, t 
```


1. Construa um SFOTS usando BitVector’s de tamanho $n=16\,$ bits que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
2. Usando a metodologia das  “Constraint Horn Clauses”(chc’s) verifique se é possível determinar um invariante que garanta que nunca se atinge um estado de erro.
3. Verifique, usando a metodologia dos invariantes e interpolantes, se o modelo atinge um estado de erro. 
Para o cálculo do interpolante usar a metodologia das “Constraint Horn Clauses”(chc’s).



### exercicio 1. e 2. sem o overflow de bits



In [41]:
from z3 import *

def check_exa_chc():

    N = 16

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)  #rb é o r_ do enunciado
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

  
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    #SFOTS
    init = And(
        a > 0, b > 0,
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )


    q = r / rb 

    

    # t0 é o começo do loop
    t0 = And(on, rb != 0,         
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,              # Todo o calculo 
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on        # As variaveis que nao mudam 
    )

    # saida do loop
    t1 = And( on,rb == 0,          
        
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb, # As variaveis que nao mudam 
        a_p == a, b_p == b,
        
        on_p == Not(on)   # o loop acabou
    )

    trans = Or(t0, t1)

    # ERROR
    error = And(on, r == 0)

    
    Inv = Function('inv',  BitVecSort(N), BitVecSort(N), BitVecSort(N),
    BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N),BoolSort(), 
    BoolSort())

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)


    chc0 = ForAll(vars_curr, Implies(init, pre))

    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))

    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))


    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança (r != 0)...")
    result = s.check()
    
    if result == sat:
        print("SAFE! O programa é seguro (r nunca é 0 indevidamente).")
        print(s.model().eval(pre)) # Pode imprimir o invariante descoberto
    elif result == unsat:
        print("UNSAFE! Encontrado contra-exemplo.")
    else:
        print("UNKNOWN.")

check_exa_chc()

A verificar segurança (r != 0)...
SAFE! O programa é seguro (r nunca é 0 indevidamente).
Not(r == 0)


### Agora admitindo o erro do overflow:

In [42]:
from z3 import *

def check_exa_chc():

    N = 4

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)  #rb é o r_ do enunciado
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

  
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    #SFOTS
    init = And(
        a > 0, b > 0, 
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )


    q = UDiv(r, rb)
   

    def mul_overflow(x, y):
        return Not(BVMulNoOverflow(x, y, True))

    def sub_overflow(x, y):
        return Or(
            Not(BVSubNoOverflow(x, y)),
            Not(BVSubNoUnderflow(x, y, True))
        )

    mult_rnext_overflow = mul_overflow(q, rb)
    mult_snext_overflow = mul_overflow(q, sb)
    mult_tnext_overflow = mul_overflow(q, tb)

    sub_r_overflow = sub_overflow(r, q * rb)
    sub_s_overflow = sub_overflow(s, q * sb)
    sub_t_overflow = sub_overflow(t, q * tb)

    hasOverFlow = And(on, 
        rb != 0,Or(
        mult_rnext_overflow,
        mult_snext_overflow,
        mult_tnext_overflow,
        sub_r_overflow,
        sub_s_overflow,
        sub_t_overflow
    )
    )




    # t0 é o começo do loop
    t0 = And(on, rb != 0,         
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,              # Todo o calculo 
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on        # As variaveis que nao mudam 
    )

    # saida do loop
    t1 = And( on,rb == 0,          
        
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb, # As variaveis que nao mudam 
        a_p == a, b_p == b,
        
        on_p == Not(on)   # o loop acabou
    )

    trans = Or(t0, t1)

    # ERROR
    error = Or(
        hasOverFlow,
        And(Not(on), r == 0) 
    )

    

    # funçao inv que tem as variaveis ints e o bool do loop e retorna um bool
    Inv = Function('inv',  BitVecSort(N), BitVecSort(N), BitVecSort(N),
    BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N),BoolSort(), 
    BoolSort())

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    # CHC 1: Init => Invariante
    chc0 = ForAll(vars_curr, Implies(init, pre))

    # CHC 2: (Invariante AND Transição) => Invariante'
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))

    # CHC 3: (Invariante AND Erro) => False (Absurdo)
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))


    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança e a procurar um invariante")
    result = s.check()
    
    if result == sat:
        print("SAFE! O programa é seguro")
        print(s.model().eval(pre)) 
    elif result == unsat:
        print("UNSAFE!!!")
        #print(s.proof())
    else:
        print("UNKNOWN...")
        print(s.reason_unknown())

check_exa_chc()

A verificar segurança e a procurar um invariante
SAFE! O programa é seguro
And(Or(Not(tb*
           bvudiv_i(r, rb)*
           bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
           tb +
           15*t*bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) <=
           0),
       rb*bvudiv_i(r, rb) == r,
       tb*
       bvudiv_i(r, rb)*
       bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
       15*t*bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) <=
       0),
    Or(sb*
       bvudiv_i(15*rb*bvudiv_i(r, rb) + r,
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
                15*r*bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
                rb) +
       15*
       s*
       bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r)*
       bvudiv_i(15*rb*bvudiv_i(r, rb) + r,
                rb*
                bvudiv_i(r, rb)*
                bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
                15*r*bvudiv_i(rb, 15*rb*bvudiv_i(r, rb) + r) +
                rb) +

### Calculo dos interpolantes

In [ ]:
from z3 import *

def free_vars(fml):
    seen = set([]) ; vars = set([]) 
    def fv(seen, vars, f):
        if f in seen: return
        seen |= { f }
        if is_const(f) and f.decl().kind() == Z3_OP_UNINTERPRETED:
            vars |= { f }
        for ch in f.children():
            fv(seen, vars, ch)
    fv(seen, vars, fml)
    return vars

def interpolate(A, B):
    As = free_vars(A)
    Bs = free_vars(B)
    shared = [s for s in As & Bs]
    
   
    if not shared: return BoolVal(False)

    
    Itp = Function('Itp', [s.sort() for s in shared] + [BoolSort()])
    C  = Itp(shared)
    
    chc0 = ForAll([a for a in As], Implies(A, C))
    chc1 = ForAll([b for b in Bs], Implies(And(C, B), BoolVal(False)))
    
    solver = SolverFor('HORN')
    solver.add([chc0, chc1])
    if solver.check() == sat:
        return solver.model().eval(C)
    return None




def check_exa_chc_interpolantes():
    N = 4

    # Variáveis Atuais (k=0)
    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)
    s, sb   = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

    # Variáveis Próximas (k=1)
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    # Listas de variáveis
    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]

   
    q = UDiv(r, rb) 

    

    def mul_overflow(x, y):
        return Not(BVMulNoOverflow(x, y, True))

    def sub_overflow(x, y):
        return Or(
            Not(BVSubNoOverflow(x, y)),
            Not(BVSubNoUnderflow(x, y, True))
        )

    mult_rnext_overflow = mul_overflow(q, rb)
    mult_snext_overflow = mul_overflow(q, sb)
    mult_tnext_overflow = mul_overflow(q, tb)

    sub_r_overflow = sub_overflow(r, q * rb)
    sub_s_overflow = sub_overflow(s, q * sb)
    sub_t_overflow = sub_overflow(t, q * tb)

    hasOverFlow = Or(
        mult_rnext_overflow,
        mult_snext_overflow,
        mult_tnext_overflow,
        sub_r_overflow,
        sub_s_overflow,
        sub_t_overflow
    )

    # --- FÓRMULAS DO SISTEMA ---

    # 1. INIT (Sobre vars_curr)
    init = And(
        a > 0, b > 0,
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )

    # 2. TRANSIÇÃO (vars_curr -> vars_next)
    # Recalcular q para a transição
    q_curr = UDiv(r, rb) 
    
    t0 = And(on, rb != 0,
        r_p == rb,
        rb_p == r - (q_curr * rb),
        s_p == sb,
        sb_p == s - (q_curr * sb),
        t_p == tb,
        tb_p == t - (q_curr * tb),
        a_p == a, b_p == b,
        on_p == on
    )

    t1 = And(on, rb == 0,
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb,
        a_p == a, b_p == b,
        on_p == Not(on)
    )

    trans = Or(t0, t1)

    
  
    
    # O erro é: Ocorreu Overflow na transição OU o resultado r_p é 0 (estando on)
    error = Or(hasOverFlow, And(on, r == 0))


    
    # A 
    Formula_A = And(init, trans) 
    
    # B 
    Formula_B = error 

    I = interpolate(Formula_A, Formula_B)

    if I is not None:
        print(I)


check_exa_chc_interpolantes()

Exists([k!10, k!9, k!8, k!7, k!5, k!4, k!3, k!1, k!0],
       And(Not(rb <= 0),
           Or(And(Not(k!3),
                  k!8 == rb,
                  rb == 0,
                  k!5 == 1,
                  k!10 == 0,
                  k!1 == r,
                  k!0 == r,
                  k!4 == 1,
                  k!9 == rb,
                  k!7 == 0),
              And(k!3,
                  k!8 == r + 15*rb*bvudiv_i(r, rb),
                  Not(rb == 0),
                  k!5 == 0,
                  k!10 == 1,
                  k!1 == rb,
                  k!0 == r,
                  k!4 == 15*bvudiv_i(r, rb),
                  k!9 == rb,
                  k!7 == 1)),
           Not(r <= 0),
           sb == 0,
           on,
           t == 0,
           tb == 1,
           s == 1))
